## 读取二维门店表

In [1]:
import pandas as pd
import pyodbc
import numpy as np
from sqlalchemy import create_engine
from tqdm import tqdm_notebook as tqdm  
import warnings
import re
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment,Font,PatternFill
from openpyxl.cell.text import InlineFont
from openpyxl.cell.rich_text import TextBlock, CellRichText
from IPython.core.display import display, HTML
import itertools
warnings.filterwarnings("ignore")

file_paths = {
    '整体': r"C:\Users\hp\Desktop\8.27周会BI导出表\整体8.27.xlsx",
    '2年+店': r"C:\Users\hp\Desktop\8.27周会BI导出表\2年+店8.27.xlsx",
    '同业可比店': r"C:\Users\hp\Desktop\8.27周会BI导出表\同业可比店8.27.xlsx",
    '门店': r"C:\Users\hp\Desktop\8.27周会BI导出表\门店8.27.xlsx",
    '事业部业绩': r"C:\Users\hp\Desktop\8.27周会BI导出表\事业部业绩战报（仅统计可比战区)8.27.xlsx",
    '同比业绩':r"C:\Users\hp\Desktop\8.27周会BI导出表\整体8.20.xlsx",
    '商品通用名为负':r"C:\Users\hp\Desktop\8.27周会BI导出表\商品通用名同比值为负——到战区8.27.xlsx",
    '战区通用名':r"C:\Users\hp\Desktop\8.27周会BI导出表\通用名TOP100-战区（大数据）8.27.xlsx",
    'O2O业绩':r"C:\Users\hp\Desktop\8.27周会BI导出表\自营品类业绩O2O-8.27.xlsx",
    '客流监控整体数据':r"C:\Users\hp\Desktop\8.27周会BI导出表\客流监控明细-到战区8.27（整体）.xlsx",
    '客流监控两年+数据':r"C:\Users\hp\Desktop\8.27周会BI导出表\客流监控明细-到战区（2年+店8.27）.xlsx"
}
org_dict = {11003873: '通辽战区',10013000: '安徽战区',90001030: '赤峰战区',10011000: '广西战区',91000001: '湘西战区',
 10002000: '湖南战区',90001031: '渭南公司',11003318: '甘宁战区',11003525: '南通战区',10015000: '苏南战区',11007532: '运城战区',
 10012000: '陕西战区',10004000: '天津战区',10003000: '新中战区',11005118: '临沂战区',10008000: '新东战区',11005157: '江阴战区',
 11002580: '扬州战区'}

C:\Users\hp\AppData\Local\Temp\ipykernel_19612\181467888.py:13: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
def process_2vexcel_file(filepath, header_levels):
    # Create a dynamic header list based on header_levels
    header_list = list(range(header_levels))
    df = pd.read_excel(filepath, header=header_list)
    def remove_unnamed(multiindex):
        return [(level if "Unnamed:" not in level else '') for level in multiindex]
    all_columns = [remove_unnamed(col) for col in df.columns]
    flattened_columns = ['_'.join(filter(None, col)) for col in all_columns]
    df.columns = flattened_columns
    df.replace('-', np.nan, inplace=True)
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='ignore')  # If conversion fails, retain original value
    numeric_columns = df.select_dtypes(include=[float, int]).columns
    df[numeric_columns] = df[numeric_columns].round(4)
    return df
df = process_2vexcel_file(file_paths['门店'],header_levels=2)
zones = list(df['战区'].unique())
zones.insert(0, '自营直营')

## 请货满足率、商品断货率、事业部、毛利信息、战区核心城市公司毛利达成率

In [3]:
def process_and_extract_columns(file_path, sheet_index, columns):
    df = process_2vexcel_file(file_path, sheet_index)
    extracted = df[columns]
    return extracted
# 这里是您之前提供的函数，我这里仅作为占位符，真实代码中应该保持不变
def filter_data_by_shop_type(zone_df, shop_type):
    """Filter data based on shop_type criteria."""
    if shop_type == '整体':
        return zone_df
    else:  # 为了处理所有其他类型，包括'2年+店'和'同业可比店'
        return zone_df[zone_df['店龄店型'] == shop_type]
def percent_format(df, cols):
    for col in cols:
        df[col] = (df[col] * 100).round(1).astype(str) + '%'
    return df

def calculate_city_margin_achievement(data):
    valid_data = data.dropna(subset=['补偿后毛利（万元）_实绩', '补偿后毛利（万元）_实时目标'])
    valid_data = valid_data[(valid_data['补偿后毛利（万元）_实绩'] != 0) & (valid_data['补偿后毛利（万元）_实时目标'] != 0)]
    if valid_data['补偿后毛利（万元）_实时目标'].sum() == 0:
        return "-"
    achievement_rate = (valid_data['补偿后毛利（万元）_实绩'].sum() / valid_data['补偿后毛利（万元）_实时目标'].sum()) * 100
    return f"{achievement_rate:.1f}%"

def format_percentage(x):
    if pd.isnull(x):  # Check for NaN
        return x
    return "{:.1f}%".format(x * 100)
# 根据百分比降序排序城市毛利达成率
def sort_achievement_by_percentage(x):
    items = x.split('<br><br>')

    def get_value(item):
        percentage = item.split('：')[-1].replace('%', '')
        return float('-inf') if percentage == '-' else float(percentage)
        
    sorted_items = sorted(items, key=get_value, reverse=True)
    return '<br><br>'.join(sorted_items)
# 根据百分比降序排序事业部毛利信息
def sort_margin_by_percentage(x):
    return sort_achievement_by_percentage(x)

def add_shop_type_column(df, shop_type):
    df['店龄店型'] = shop_type
    return df
# 为每个shop_type创建一个dataframe

shop_type_dfs = {key: process_2vexcel_file(value, 3) for key, value in itertools.islice(file_paths.items(), 3)}

result_df = pd.DataFrame(columns=['战区', '店龄店型', '达成率', '同比率', '毛利率同比'])
zones = shop_type_dfs['整体']['战区'].unique()

for zone in zones:
    for shop_type, df in shop_type_dfs.items():
        zone_df = df[df['战区'] == zone]
        filtered_df = filter_data_by_shop_type(zone_df, shop_type)

        if not filtered_df.empty:
            result_df = result_df.append({
                '战区': zone,
                '店龄店型': shop_type,
                '达成率': filtered_df['补偿后毛利额（万元）_本月实时达成（当月1日截止至昨日）_达成率'].iloc[0],
                '同比率': filtered_df['补偿后毛利额（万元）_本月实时达成（当月1日截止至昨日）_同比率'].iloc[0],
                '毛利率同比': filtered_df['自营前台含税毛利率_同比率'].iloc[0]
            }, ignore_index=True)   
            


df_4 = process_and_extract_columns(file_paths['事业部业绩'], 3, ['战区', '业务板块', '事业部', '同比情况（万元）_同比率_补偿后毛利额'])
df_4 = df_4.rename(columns={'业务板块': '店龄店型'})
df_4 = df_4[df_4['事业部'] != '合计']
df_4 = percent_format(df_4, ['同比情况（万元）_同比率_补偿后毛利额'])
df = process_2vexcel_file(file_paths['门店'],header_levels=2)

# Calculating city margin achievement
# 计算“整体”
city_margin_achievement = df.groupby(['战区', '城市公司']).apply(calculate_city_margin_achievement).reset_index()
city_margin_achievement.columns = ['战区', '城市公司', '达成率']
formatted_rates_overall = city_margin_achievement.groupby('战区').apply(lambda x: '<br><br>'.join(x['城市公司'] + "：" + x['达成率'])).reset_index()
formatted_rates_overall.columns = ['战区', '战区核心城市公司毛利达成率']
formatted_rates_overall['战区核心城市公司毛利达成率'] = formatted_rates_overall['战区核心城市公司毛利达成率'].apply(sort_achievement_by_percentage)
formatted_rates_overall['店龄店型'] = '整体'

# 计算“2年+店”
df_2_years_plus = df[df['店龄店型'] == '2年+店']
df_2_years_plus = df_2_years_plus.groupby(['战区', '城市公司']).apply(calculate_city_margin_achievement).reset_index()
df_2_years_plus.columns = ['战区', '城市公司', '达成率']
formatted_rates_df_2_years_plus = df_2_years_plus.groupby('战区').apply(lambda x: '<br><br>'.join(x['城市公司'] + "：" + x['达成率'])).reset_index()
formatted_rates_df_2_years_plus.columns = ['战区', '战区核心城市公司毛利达成率']
formatted_rates_df_2_years_plus['战区核心城市公司毛利达成率'] = formatted_rates_df_2_years_plus['战区核心城市公司毛利达成率'].apply(sort_achievement_by_percentage)
formatted_rates_df_2_years_plus['店龄店型'] = '2年+店'

# 计算“同业可比店”
df_comparable = df[df['店龄店型'].isin(['2年店', '2年+店'])]
df_comparable = df_comparable.groupby(['战区', '城市公司']).apply(calculate_city_margin_achievement).reset_index()
df_comparable.columns = ['战区', '城市公司', '达成率']
formatted_rates_df_comparable = df_comparable.groupby('战区').apply(lambda x: '<br><br>'.join(x['城市公司'] + "：" + x['达成率'])).reset_index()
formatted_rates_df_comparable.columns = ['战区', '战区核心城市公司毛利达成率']
formatted_rates_df_comparable['战区核心城市公司毛利达成率'] = formatted_rates_df_comparable['战区核心城市公司毛利达成率'].apply(sort_achievement_by_percentage)
formatted_rates_df_comparable['店龄店型'] = '同业可比店'
# 将这三个DataFrames合并成一个
formatted_rates = pd.concat([formatted_rates_overall, formatted_rates_df_2_years_plus, formatted_rates_df_comparable], axis=0).reset_index(drop=True)

grouped = df_4.groupby(['战区', '店龄店型']).apply(lambda x: '<br><br>'.join(x['事业部'] + '：' + x['同比情况（万元）_同比率_补偿后毛利额'])).reset_index()
grouped.columns = ['战区', '店龄店型', '事业部及毛利信息']
grouped['事业部及毛利信息'] = grouped['事业部及毛利信息'].apply(sort_margin_by_percentage)
shop_types = ['整体', '2年+店', '同业可比店']
cols_needed = ['战区', '自营直营门店请货满足率_实绩', '自营直营全量商品断货率_实绩']
dfs = [add_shop_type_column(process_and_extract_columns(value, 3, cols_needed), key) 
       for key, value in itertools.islice(file_paths.items(), 3)]
df_total = pd.concat(dfs)
df_total = percent_format(df_total, cols_needed[1:])
result = result_df.merge(df_total, on=['战区', '店龄店型'], how='left')
result = result.merge(formatted_rates, on=['战区','店龄店型'], how='left')

merged_df = result.merge(grouped, on=['战区', '店龄店型'], how='left')
merged_df.set_index(['战区', '店龄店型'], inplace=True)
merged_df[['达成率', '同比率']] = merged_df[['达成率', '同比率']].applymap(format_percentage)
merged_df = merged_df.reset_index()
tb = process_2vexcel_file(file_paths['同比业绩'],3)[['战区', '店龄店型','补偿后毛利额（万元）_本月实时达成（当月1日截止至昨日）_达成率']]
merged_df = merged_df.merge(tb, on=['战区', '店龄店型'], how='left')
merged_df['达成率'] = merged_df['达成率'].str.rstrip('%').astype('float') / 100.0
merged_df['上次业绩追踪达成率环比'] = (merged_df['达成率'] - merged_df['补偿后毛利额（万元）_本月实时达成（当月1日截止至昨日）_达成率']) * 100
merged_df['上次业绩追踪达成率环比'] = merged_df['上次业绩追踪达成率环比'].apply(lambda x: "{:.1f}pp".format(x))
merged_df.set_index(['战区', '店龄店型'], inplace=True)
# 输出
display(HTML(merged_df.to_html(escape=False)))

## 商品通用名、O2O达成率

In [6]:
minus_value = pd.read_excel(file_paths['商品通用名为负']).replace({'lev3_org_id': org_dict}).rename(columns={'lev3_org_id':'战区','sale_diff':'负值总额'})  # 映射ID到中文名
minus_value = minus_value.set_index('战区')
def get_top_5_goods(group):
    # 检查商品列表中是否有nan，如果有则提取更多的商品
    goods_list = list(group['商品通用名'].astype(str))  # 确保所有的商品名都是字符串
    if 'nan' in goods_list:
        additional_goods = group.nlargest(10, '排名')['商品通用名'].astype(str).tolist()  # 同样确保这些也是字符串
        goods_list = [goods for goods in additional_goods if goods != 'nan'][:5]
    else:
        goods_list = goods_list[:5]
    return '<br><br>'.join(goods_list)

def compute_percentage(row):
    product_info = row['商品通用名销售额同比值下降TOP5商品']
    
    # 检查是否是字符串类型，如果不是则直接返回原值
    if not isinstance(product_info, str):
        return product_info

    # 提取商品名中的数值
    values = re.findall(r"(-\d+\.\d+)", product_info)
    
    # 对每一个数值进行计算
    computed_values = []
    for val in values:
        val_float = float(val) * 10000  # 转换为原始数值
        computed_val = val_float / minus_value.loc[row['战区'], '负值总额']
        percentage_val = f"{computed_val*100:.1f}%"
        computed_string = f"{val}万,{percentage_val}"
        product_info = product_info.replace(f"{val}万", computed_string)

    return product_info

df5 = (
    pd.read_excel(file_paths['战区通用名'])[['lev3_org_id', 'goods_common_name', 'rn', 'sale_diff']]
    .replace({'lev3_org_id': org_dict})  # 映射ID到中文名
    .rename(columns={
        'lev3_org_id': '战区',
        'goods_common_name': '商品通用名',
        'rn': '排名'
    })  # 重命名列
    .assign(
        sale_diff=lambda df: (df['sale_diff'] / 10000).round(1),
        商品通用名=lambda df: df['商品通用名'] + '(' + df['sale_diff'].astype(str) + '万)'
    )
)

# 直接筛选排名在1-5的商品并聚合
top_5_string = (
    df5[df5['排名'].isin(range(1, 11))]  # 先选取前10的商品，确保有足够的商品替换nan
    .groupby('战区')
    .apply(get_top_5_goods)
    .reset_index()
    .rename(columns={0: '商品通用名销售额同比值下降TOP5商品'})
)
top_5_string['店龄店型'] ='整体'
# 读取并处理O2O数据
o2o = (
    process_2vexcel_file(file_paths['O2O业绩'], 3)[['战区', '本期情况（万元）_本月实时达成（本月1日截止至昨日）_销售额']]
    .rename(columns={'本期情况（万元）_本月实时达成（本月1日截止至昨日）_销售额': 'O2O销售额达成率'})
    .assign(O2O销售额达成率=lambda df: (df['O2O销售额达成率'] * 100).round(1).astype(str) + '%')
)

# 合并数据并显示
final_df = (
    merged_df.reset_index()
    .merge(top_5_string, on=['战区','店龄店型'], how='left')
    .merge(o2o, on='战区', how='left')
    .set_index(['战区', '店龄店型', '商品通用名销售额同比值下降TOP5商品'])
)

final_df = final_df.reset_index()
final_df['商品通用名销售额同比值下降TOP5商品'] = final_df.apply(compute_percentage, axis=1)
final_df = final_df.set_index(['战区', '店龄店型'])
display(HTML(final_df.to_html(escape=False)))

## 客流数据

In [7]:
wl = process_2vexcel_file(file_paths['客流监控整体数据'],3)[['部门','店前客流(店日均)_本期','进店率_本期','进店率_同比','成交率_本期','成交率_同比',
                                                                '转化率_本期']]
wl.loc[0, '部门'] = '自营直营'
# 将百分比字符串转换为浮点数
def percentage_to_float(percentage_str):
    return float(percentage_str.strip('%')) / 100

def percentage_to_float(value):
    if isinstance(value, str):
        return float(value.strip('%')) / 100
    elif isinstance(value, (int, float)):
        return value
    else:
        raise ValueError(f"Unexpected value: {value}")
# 对'成交率_本期'和'成交率_同比'应用转换函数
wl['成交率_本期'] = wl['成交率_本期'].apply(percentage_to_float)
wl['成交率_同比'] = wl['成交率_同比'].apply(percentage_to_float)
# 计算额外的列
wl['本期进店客数'] = wl['店前客流(店日均)_本期'] * wl['进店率_本期']
wl['本期成交客数'] = wl['本期进店客数'] * wl['成交率_本期']
wl['同期成交客数'] = wl['本期进店客数'] * (wl['成交率_本期'] - wl['成交率_同比'])
wl['成交客数同比'] = (wl['本期成交客数'] / wl['同期成交客数']) - 1
# 格式化成百分比和pp格式
wl['成交客数同比'] = (wl['成交客数同比'] * 100).round(1).astype(str) + '%'
wl['进店率_同比'] = (wl['进店率_同比'] * 100).round(1).astype(str) + 'pp'
wl['成交率_同比'] = (wl['成交率_同比'] * 100).round(1).astype(str) + 'pp'
# 从原始的 wl DataFrame 中选择所需的列，并重命名它们
new_df = wl[['部门', '进店率_本期', '进店率_同比', '本期进店客数', '成交率_本期', '成交率_同比', '本期成交客数', '成交客数同比']].copy()
new_df.columns = ['战区', '进店率', '进店率同比', '进店客数（店日均）', '成交率', '成交率同比', '成交客数（店日均）', '成交客数同比']
new_df['店龄店型'] = '整体'

# 导入新文件
wl_new = process_2vexcel_file(file_paths['客流监控两年+数据'],3)[['部门','店前客流(店日均)_本期','进店率_本期','进店率_同比','成交率_本期','成交率_同比','转化率_本期']]
wl_new.loc[0, '部门'] = '自营直营'
# 使用上面的percentage_to_float函数进行转换
wl_new['成交率_本期'] = wl_new['成交率_本期'].apply(percentage_to_float)
wl_new['成交率_同比'] = wl_new['成交率_同比'].apply(percentage_to_float)

# 计算额外的列
wl_new['本期进店客数'] = wl_new['店前客流(店日均)_本期'] * wl_new['进店率_本期']
wl_new['本期成交客数'] = wl_new['本期进店客数'] * wl_new['成交率_本期']
wl_new['同期成交客数'] = wl_new['本期进店客数'] * (wl_new['成交率_本期'] - wl_new['成交率_同比'])
wl_new['成交客数同比'] = (wl_new['本期成交客数'] / wl_new['同期成交客数']) - 1

# 格式化为百分比和pp格式
wl_new['成交客数同比'] = (wl_new['成交客数同比'] * 100).round(1).astype(str) + '%'
wl_new['进店率_同比'] = (wl_new['进店率_同比'] * 100).round(1).astype(str) + 'pp'
wl_new['成交率_同比'] = (wl_new['成交率_同比'] * 100).round(1).astype(str) + 'pp'

# 从新的 wl_new DataFrame 中选择所需的列，并重命名它们
new_df_2 = wl_new[['部门', '进店率_本期', '进店率_同比', '本期进店客数', '成交率_本期', '成交率_同比', '本期成交客数', '成交客数同比']].copy()
new_df_2.columns = ['战区', '进店率', '进店率同比', '进店客数（店日均）', '成交率', '成交率同比', '成交客数（店日均）', '成交客数同比']

# 为新的DataFrame添加'店龄店型'列
new_df_2['店龄店型'] = '2年+店'

tb1 = process_2vexcel_file(file_paths['整体'],3)[['战区', '店龄店型','成交客数（线上+线下）_同比率']]
tb2 = process_2vexcel_file(file_paths['同业可比店'],3)[['战区', '店龄店型','成交客数（线上+线下）_同比率']]
tb3 = process_2vexcel_file(file_paths['2年+店'],3)[['战区', '店龄店型','成交客数（线上+线下）_同比率']]
tb4 = pd.concat([tb1,tb2,tb3])

# 将两个DataFrame合并
new_df = pd.concat([new_df,new_df_2])
new_df = new_df.drop('成交客数同比',axis=1)
new_df = new_df.merge(tb4, on=['战区', '店龄店型'], how='outer')

final_df = final_df.reset_index()
final_df = final_df.merge(new_df, on=['店龄店型','战区'], how='outer')
final_df.set_index(['战区', '店龄店型'], inplace=True)
final_df.replace(["nan%", "nanpp"], pd.NA, inplace=True)

final_df["进店客数（店日均）"].fillna(-1, inplace=True)
final_df["进店客数（店日均）"] = final_df["进店客数（店日均）"].astype(int)
final_df["进店客数（店日均）"] = final_df["进店客数（店日均）"].where(final_df["进店客数（店日均）"] != -1, pd.NA)
final_df["成交客数（店日均）"].fillna(-1, inplace=True)
final_df["成交客数（店日均）"] = final_df["成交客数（店日均）"].astype(int)
final_df["成交客数（店日均）"] = final_df["成交客数（店日均）"].where(final_df["成交客数（店日均）"] != -1, pd.NA)
final_df['成交客数（线上+线下）_同比率'] = (final_df['成交客数（线上+线下）_同比率'] * 100).round(1).astype(str) + '%'
final_df["进店率"] = (final_df["进店率"] * 100).round(1).astype(str) + "%"
final_df["成交率"] = (final_df["成交率"] * 100).round(1).astype(str) + "%"
final_df["达成率"] = (final_df["达成率"] * 100).round(1).astype(str) + "%"
final_df.replace(["nan%", "nanpp"], pd.NA, inplace=True)
display(HTML(final_df.to_html(escape=False)))

## 保存为excel文档

In [8]:
final_df.reset_index(inplace=True)
final_df = (final_df[['战区','店龄店型','达成率','同比率','毛利率同比','上次业绩追踪达成率环比','战区核心城市公司毛利达成率',
                     '进店率','进店率同比', '进店客数（店日均）','成交率','成交率同比','成交客数（店日均）', '成交客数（线上+线下）_同比率',
                     '自营直营全量商品断货率_实绩','自营直营门店请货满足率_实绩','事业部及毛利信息','商品通用名销售额同比值下降TOP5商品',
                     'O2O销售额达成率']]).rename(
    columns={'事业部及毛利信息':'事业部补偿后毛利额同比','自营直营全量商品断货率_实绩':'自营直营全量商品断货率',
                                     '自营直营门店请货满足率_实绩':'自营直营门店请货满足率'})
final_df.set_index(['战区', '店龄店型'],inplace=True)

In [14]:
index_fill = PatternFill(start_color="36A3E7", end_color="36A3E7", fill_type="solid")
light_gray_fill = PatternFill(start_color="CADCE8", end_color="CADCE8", fill_type="solid")
lighter_blue_fill = PatternFill(start_color="C7EBFF", end_color="C7EBFF", fill_type="solid")
index_font = Font(color="FFFFFF", bold=False)
red_font = InlineFont(color="FF0000")
green_font = InlineFont(color="008000")  # 绿色
sql_data = final_df.replace('<br><br>', '\n', regex=True).replace(np.NAN,'-', regex=True).reset_index()
# 创建一个新的工作簿并获取活动工作表
wb = Workbook()
ws = wb.active
LENGTH_THRESHOLD = 30
ADDITIONAL_HEIGHT_PER_THRESHOLD = 6
SPECIFIC_ROW_HEIGHT = 60
# Adjust the rows based on your code with specific considerations for the mentioned zones
for r_idx, row in enumerate(dataframe_to_rows(sql_data, index=False, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        cell = ws.cell(row=r_idx, column=c_idx, value=value)
        cell.alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)

        # Set a fixed height for the header row
        if r_idx == 1:
            ws.row_dimensions[r_idx].height = 45
            ws.row_dimensions[r_idx].width = 45
            cell.font = index_font
            cell.fill = index_fill
        # Adjust row height based on content for the specific columns
        zone_value = ws.cell(row=r_idx, column=sql_data.columns.get_loc("战区") + 1).value
        if c_idx == sql_data.columns.get_loc("商品通用名销售额同比值下降TOP5商品") + 1 and zone_value != '自营直营':  # +1 because Excel is 1-based
            num_of_lines = value.count('\n') + 1
            additional_height = 0
            if len(value) > LENGTH_THRESHOLD:
                additional_height = (len(value) // LENGTH_THRESHOLD) * ADDITIONAL_HEIGHT_PER_THRESHOLD
            ws.row_dimensions[r_idx].height = (15 * num_of_lines) + additional_height
        if c_idx == sql_data.columns.get_loc("事业部补偿后毛利额同比") + 1:
            num_of_lines = value.count('\n') + 1
            additional_height = 0
            if len(value) > LENGTH_THRESHOLD:
                additional_height = (len(value) // LENGTH_THRESHOLD) * ADDITIONAL_HEIGHT_PER_THRESHOLD
            ws.row_dimensions[r_idx].height = (22 * num_of_lines) + additional_height
        current_width = ws.column_dimensions[chr(64 + c_idx)].width or 0
        required_width = len(str(value))
        if c_idx == sql_data.columns.get_loc("商品通用名销售额同比值下降TOP5商品") + 1:
            ws.column_dimensions[chr(64 + c_idx)].width = 35
        elif c_idx == sql_data.columns.get_loc("战区核心城市公司毛利达成率") + 1:
            ws.column_dimensions[chr(64 + c_idx)].width = 25
        elif c_idx == sql_data.columns.get_loc("事业部补偿后毛利额同比") + 1:
            ws.column_dimensions[chr(64 + c_idx)].width = 25
        # Adjust row height for specific zones
        if c_idx == sql_data.columns.get_loc("战区") + 1 and value in ["扬州战区", "天津战区", "赤峰战区"]:
            ws.row_dimensions[r_idx].height = SPECIFIC_ROW_HEIGHT
        if c_idx == sql_data.columns.get_loc("战区") + 1 and value == '自营直营':
            ws.row_dimensions[r_idx].height = 30
            continue
        if c_idx == sql_data.columns.get_loc("商品通用名销售额同比值下降TOP5商品") + 1 :
            rich_text = CellRichText()
            last_end = 0
            # 使用finditer找到所有匹配项
            for match in re.finditer(r"(-\d+\.\d+)万", str(value)):
                num_value = float(match.group(1))
                if num_value < 0:
                    # 获取匹配的开始和结束位置
                    start, end = match.span(1)
                    # 添加匹配前的文本
                    rich_text.append(str(value)[last_end:start])
                    # 以红色添加匹配到的数值
                    rich_text.append(TextBlock(red_font, str(value)[start:end]))
                    last_end = end
            # 添加最后一个匹配后的文本
            rich_text.append(str(value)[last_end:])
            cell.value = rich_text  
        if c_idx in [sql_data.columns.get_loc("战区核心城市公司毛利达成率") + 1, sql_data.columns.get_loc("事业部补偿后毛利额同比") + 1]:
            rich_text = CellRichText()
            last_end = 0
            pattern = r"(-?\d+\.\d+)%" if c_idx == sql_data.columns.get_loc("事业部补偿后毛利额同比") + 1 else r"(\d+\.\d+)%"
            for match in re.finditer(pattern, str(value)):
                percentage_value = float(match.group(1))
                start, end = match.span(1)
                if c_idx == sql_data.columns.get_loc("战区核心城市公司毛利达成率") + 1:
                    if percentage_value > 100:
                        selected_font = green_font
                    elif percentage_value < 80:
                        selected_font = red_font
                    else:
                        selected_font = None
                else:  # 事业部补偿后毛利额同比
                    if percentage_value < 0:
                        selected_font = red_font
                    else:
                        selected_font = None
                rich_text.append(str(value)[last_end:start])
                if selected_font:
                    rich_text.append(TextBlock(selected_font, str(value)[start:end]))
                else:
                    rich_text.append(str(value)[start:end])
                last_end = end
            rich_text.append(str(value)[last_end:])
            cell.value = rich_text

        if c_idx in [sql_data.columns.get_loc("达成率") + 1, sql_data.columns.get_loc("同比率") + 1, 
                     sql_data.columns.get_loc("自营直营全量商品断货率") + 1, sql_data.columns.get_loc("O2O销售额达成率") + 1,
                    sql_data.columns.get_loc("成交客数（线上+线下）_同比率") + 1,sql_data.columns.get_loc("自营直营门店请货满足率") + 1]:
            # 提取百分比
            match = re.search(r"(-?\d+\.\d+)%", str(value))
            if match:
                percentage_value = float(match.group(1))       
                # 根据不同的列和百分比的大小选择颜色
                if c_idx == sql_data.columns.get_loc("达成率") + 1:
                    if percentage_value > 100:
                        selected_font = green_font
                    elif percentage_value < 80:
                        selected_font = red_font
                    else:
                        selected_font = None
                elif c_idx == sql_data.columns.get_loc("同比率") + 1:
                    if percentage_value > 10:
                        selected_font = green_font
                    elif percentage_value <= -5:
                        selected_font = red_font
                    else:
                        selected_font = None
                elif c_idx == sql_data.columns.get_loc("自营直营全量商品断货率") + 1:
                    if percentage_value > 5:
                        selected_font = red_font
                    else:
                        selected_font = None
                elif c_idx == sql_data.columns.get_loc("自营直营门店请货满足率") + 1:
                    if percentage_value < 80:
                        selected_font = red_font
                    else:
                        selected_font = None
                elif c_idx == sql_data.columns.get_loc("O2O销售额达成率") + 1:
                    if percentage_value >= 100:
                        selected_font = green_font
                    elif percentage_value < 65:
                        selected_font = red_font
                    else:
                        selected_font = None
                elif c_idx == sql_data.columns.get_loc("成交客数（线上+线下）_同比率") + 1:
                    if percentage_value >= 10:
                        selected_font = green_font
                    elif percentage_value <= -10:
                        selected_font = red_font
                    else:
                        selected_font = None
                if selected_font:
                    rich_text = CellRichText()
                    rich_text.append(TextBlock(selected_font, str(value)))
                    cell.value = rich_text
def merge_cells(column_index):
    last_value = None
    start_row = None
    for row in range(2, sql_data.shape[0] + 3):  # +3 to ensure the loop runs one step past the last row
        # Handle the last row
        if row == sql_data.shape[0] + 2:
            current_value = None
        else:
            current_value = ws.cell(row=row, column=column_index).value
        
        # Check if the current column is one of the problematic columns
        problematic_cols = [
            "自营直营门店请货满足率", 
            "自营直营全量商品断货率", 
            "事业部补偿后毛利额同比",
            '进店率同比'
        ]
        is_problematic = sql_data.columns[column_index-1] in problematic_cols

        if is_problematic:
            should_merge = ws.cell(row=row, column=sql_data.columns.get_loc("战区")+1).value == ws.cell(row=row-1, column=sql_data.columns.get_loc("战区")+1).value
        else:
            should_merge = True
        
        if current_value == last_value and current_value is not None and should_merge:
            if start_row is None:
                start_row = row - 1
        else:
            if start_row is not None:
                ws.merge_cells(start_row=start_row, start_column=column_index, end_row=row-1, end_column=column_index)
                start_row = None
        last_value = current_value

# 指定要合并的列
merge_columns = [
    "战区", "达成率", "自营直营全量商品断货率", "战区核心城市公司毛利达成率",
x    "O2O销售额达成率"]
# 合并指定的列的单元格
for col, column_name in enumerate(merge_columns, start=1):
    if col ==5 or col ==7 or col ==18: # 毛利率同比
        continue
    merge_cells(col)
merge_cells(19)
# 保存工作簿到一个新文件
output_file = "../output/merged_output_openpyxl22.xlsx"
wb.save(output_file)
print(f"File saved as {output_file}")

File saved as ../output/merged_output_openpyxl22.xlsx


## 上传数据库

sql_data = final_df.replace('<br><br>', '\n', regex=True).replace(np.NAN,'-', regex=True).reset_index()

def create_and_insert_data(table_name, df, server, database):
    # 使用sqlalchemy创建连接
    connection_string = f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes"
    engine = create_engine(connection_string)
    
    # 如果表存在，则删除
    if engine.has_table(table_name):
        with engine.connect() as conn:
            conn.execute(f"DROP TABLE {table_name}")
    
    # 使用pandas的to_sql方法插入数据，利用chunksize和tqdm显示进度条
    chunk_size = 500  # 可以根据你的需求调整
    num_chunks = (len(df) // chunk_size) + 1
    for i in tqdm(range(num_chunks), desc="插入数据"):
        df.iloc[i*chunk_size:(i+1)*chunk_size].to_sql(name=table_name, con=engine, if_exists='append', index=False)
create_and_insert_data(table_name='data_mendian', df=sql_data, server='localhost', database='data')